# Building a Feedforward Neural Network Model
This notebook provides instructions on how to build a feedforward neural network model. It will introduce a couple of classes defined in the [FFBP package](https://github.com/alex-ten/pdpyflow/tree/master/FFBP): `FFBP.Layer` and `FFBP.Model` as well as describe some of the Tensorflow's own objects needed to successfully implement a feedforward neural network model with `FFBP`.

To demonstrate various ways in which FFBP layers can be connected together, we will be constructing an arbitrary architecture featuring 3 parallel input layers (`inp_1`,`inp_2`, and `inp_3`) feeding into 2 hidden layers (`hid_1` and `hid_2`), which together send activation to a single `deep` layer, which is finally transformed by the final observable layer (`out`).

<img src="arb_net.png">

Based on these interconnected layers we will construct a working neural network and propagate test inputs through it (to see how to prepare test input for a model, consult the `prepare_input` [notebook tutorial](https://github.com/alex-ten/pdpyflow/blob/master/tutorials/FFBP_network/prepare_input.ipynb)).

We begin by importing the packages required for this tutorial and creating a [Tensorflow Graph](https://www.tensorflow.org/programmers_guide/graphs). We want to make sure that we are adding network elements in the context of this graph (see cells below).

In [ ]:
import tensorflow as tf
import FFBP

FFBP_GRAPH = tf.Graph()

## Placeholders

Input layers are implemented as [tf.placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) objects. `InputData` (covered in a [different tutorial](https://github.com/alex-ten/pdpyflow/blob/master/tutorials/building_and_training_FFBP_network/prepare_input.ipynb)) data from a text document into the `tf.float32` format. Therefore, the first argument inside the `tf.placeholder()` call is `dtype = tf.float32` (but it can be anything as long as it is consistent with the data fed in a session). The `shape` parameter is optional, but we strictly indicate *at least* the second dimension because other objects in the software expect an integer value there. The `None` in the place of the first dimension of the `shape` parameter means that any size of the that dimension is allowed. For instance, `inp_1` is a placeholder for a 2-dimensional tensor (of type `tf.float32`) with 3 columns and any number of rows. This is useful when we train a model with batched up input but test it with single examples.

The target placeholder needs to be the same size as the final (output) layer of the network. It is a good practice to name tensorflow nodes informatively, so we add an optional `name` argument accordingly.

In [ ]:
with FFBP_GRAPH.as_default():
    MODEL_NAME = 'arbitrary_model'
    with tf.name_scope(MODEL_NAME):
        
        # Create input placeholders
        INP_1  = tf.placeholder(dtype = tf.float32, shape=[None, 3], name='inp_1')
        INP_2  = tf.placeholder(dtype = tf.float32, shape=[None, 2], name='inp_2')
        INP_3  = tf.placeholder(dtype = tf.float32, shape=[None, 3], name='inp_3')
        
        # Create a target placeholder
        TARG = tf.placeholder(dtype = tf.float32, shape=[None, 8], name='targ')

## Hidden and output layers
Hidden layers are created via the `FFBP.BasicLayer` interface. Each `BasicLayer` takes the following arguments:
- **`layer_name`** : a (string) name of the layer (this will be displayed in visualization)
- **`layer_input`** : input to the layer. If input is a placeholder, just pass  the corresponding handle to this parameter. If input comes from another `BasicLayer`, we need to access its `output` attribute (e.g. `hidden_layer.output`). Finally, if there are multiple layers / placeholders feeding into a single `BasicLayer` simply wrap the inputs into a list or a tuple (e.g. `(inp1, inp2, ..., inpN)`)
- **`size`** : the number of units in the layer
- **`wrange`** : a (listed or tupled) pair of values corresponding to, respectively, the lower and upper bounds of the distribution from which weight values will be sampled uniformly upon initialization.
- **`nonlin`** : (optional, *`default`*`=None`) a function that takes in and outputs a tensor. If omitted, input to the layer will be transformed linearly (i.e. $Wx + b$).
- **`bias`** : (optional, *`default`*`=False`) controls whether to bias layer output. If ommited, the bias terms will be constant at 0.
- **`seed`** : (optional, *`default`*`=None`) the seed for random weight initialization. If omited, the initialization will be irreproducible.

These `FFBP.BasicLayer`s are modular and can be configured with different sizes, nonlinearities, weight ranges etc.

In [ ]:
with FFBP_GRAPH.as_default():
    with tf.name_scope(MODEL_NAME):

        HID_1 = FFBP.BasicLayer(
            layer_name = 'hid_1', 
            layer_input = (INP_1, INP_2), 
            size = 3, 
            wrange = [0, 3], 
            nonlin = tf.nn.sigmoid, 
            bias = True
        )
        
        HID_2 = FFBP.BasicLayer(
            layer_name = 'hid_2', 
            layer_input = INP_3, 
            size = 2, 
            wrange = [-.5, .5], 
            nonlin = None, 
            bias = False
        )
        
        DEEP = FFBP.BasicLayer(
            layer_name = 'deep', 
            layer_input = (HID_1.output, HID_2.output, INP_2), 
            size = 5, 
            wrange = [-.01, .01], 
            nonlin = tf.nn.sigmoid, 
            bias = True
        )
        
        OUT = FFBP.BasicLayer(
            layer_name = 'out', 
            layer_input = DEEP.output, 
            size = 8, 
            wrange = [-1, 1], 
            nonlin = tf.nn.softmax, 
            bias = True
        )

## FFBP Model
We can now create a feedforward model by instantiating an `FFBP.Model` class. In order to test the model later, we will need to create at least one `FFBP.InputData` object and either pass it to class initializer (`FFBP.Model.__init__()`) directly or use a separate dedicated method. An `FFBP.Model` is initialized with the following parameters:
- **`name`** : the name of the model (used for storage and visualization).
- **`loss`** : the loss (objective) function of the model.
- **`optimizer`** : model optimizer.
- **`layers`** : a list of model layers. The order is consequential for visualization. The last layer should be the output layer so that the viewer displays target output information for this layer, but not the others.
- **`inp`** : input placeholder or a list (or tuple) of input placeholders. The order of placeholders must correspond to the order of `inp_size` values set for `FFBP.InputData` (see the [input data tutorial](https://github.com/alex-ten/pdpyflow/blob/master/tutorials/building_and_training_FFBP_network/preparing_input.ipynb)).
- **`train_data`** : (optional, *`default`*`=None`) `FFBP.InputData` instance set up for training.
- **`test_data`** : (optional, *`default`*`=None`) `FFBP.InputData` instance set up for testing.



In [ ]:
with FFBP_GRAPH.as_default():
    MODEL = FFBP.Model(
        name = MODEL_NAME,
        layers = [HID_1, HID_2, DEEP, OUT], 
        inp    = [INP_1, INP_2, INP_3],
        targ   = TARG,
        loss   = tf.squared_difference(TARG, OUT.output, name='loss_function')
    )

The `FFBP.Model` class also defines a few useful methods for training and testing:
- **`test_setup`** : sets up the model for testing separately from class initialization
- **`test_epoch`** : runs a single epoch of testing by feeding each item from the test set into the model and evaluating its state. This method returns two values: the loss accumulated across test items, and the model snapshot which contains information about the model state at the time of test. The snapshots can be logged for later analyses (see the [tutorial on runlog structure](https://github.com/alex-ten/pdpyflow/blob/master/tutorials/building_models/runlog_structure.ipynb) to see how snapshot logs are organized).
- **`train_setup`** : sets up the model for training separately from class initialization
- **`train_epoch`** : runs a single epoch of training, optimizing parameters after processing each mini-batch of training examples. Returns loss accumulated over input mini-batches.

## Example: Testing simple forward propagation
Now let us test the network. By instantiating `FFBP.BasicLayer`s we implicitly added computational nodes to te underlying graph. In order to run this graph, we need to create a [session](https://www.tensorflow.org/api_docs/python/tf/Session).

In the example below we will forward-propagate the input data from `'auto_data_test.txt'` and evalute the aggregate error (total sum of squares) on the output layer by comparing it to target activations.

In [ ]:
with FFBP_GRAPH.as_default():
    
    # Define input data
    TEST_DATA = FFBP.InputData(
        path_to_data_file = 'materials/auto_data_test.txt',
        num_epochs = 2, # two epochs for pre- and post-test
        batch_size = 1,
        inp_size = [3,2,3], # sizes of INP_1, INP_2, INP_3 in order
        targ_size = 8,
        data_len = 15
    )
    
    TRAIN_DATA = FFBP.InputData(
        path_to_data_file = 'materials/auto_data_test.txt',
        num_epochs = 1,
        batch_size = 1,
        inp_size = [3,2,3], # sizes of INP_1, INP_2, INP_3 in order
        targ_size = 8,
        data_len = 15
    )
    
    # Setup testing and training
    MODEL.test_setup(TEST_DATA)
    MODEL.train_setup(TRAIN_DATA, optimizer=tf.train.GradientDescentOptimizer(learning_rate=.2))


# Run a session and compute the activation on of the output layer given inputs
with tf.Session(graph=FFBP_GRAPH) as sess:
    
    # Initialize FFBP_GRAPH variables:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())

    # create coordinator and start queue runners
    coordinator = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coordinator)
    
    # Pre-test the model
    pre_loss, pre_state = MODEL.test_epoch(session=sess, verbose=True)
    
    # Run one epoch of training
    train_loss = MODEL.train_epoch(session=sess)

    # Post-test the model
    post_loss, post_state = MODEL.test_epoch(session=sess, verbose=True)
    
    # Stop queue runners
    coordinator.request_stop()
    coordinator.join(threads)